In [1]:
import pandas as pd
import seaborn as sns
import torch
from matplotlib import pyplot as plt

import pyro
import pyro.distributions as dist
import pyro.ops.stats as stats

from rethinking import MAP, coef, extract_samples, link, precis, sim, vcov
sns.set(font_scale=1.25, rc={"figure.figsize": (10, 8)})
%config InlineBackend.figure_format = 'retina'

In [2]:
import sys
sys.path
# /Users/josephmann/anaconda3/envs/pytorch_exp/lib/python3.7/site-packages/matplotlib-3.1.1-py3.7-nspkg.pth

['/Users/josephmann/Documents/GitHub/rethinking-pyro/notebooks',
 '/Users/josephmann/anaconda3/envs/hv_pyro/lib/python37.zip',
 '/Users/josephmann/anaconda3/envs/hv_pyro/lib/python3.7',
 '/Users/josephmann/anaconda3/envs/hv_pyro/lib/python3.7/lib-dynload',
 '',
 '/Users/josephmann/anaconda3/envs/hv_pyro/lib/python3.7/site-packages',
 '/Users/josephmann/anaconda3/envs/hv_pyro/lib/python3.7/site-packages/IPython/extensions',
 '/Users/josephmann/.ipython']

In [3]:
import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import datashade, rasterize
hv.extension('bokeh') # important: doesn't render without it
hv.opts.defaults( hv.opts.Curve(width=800), hv.opts.Points(width=800),hv.opts.Histogram(width=800), hv.opts.Points(size=3))

import numpy as np

ModuleNotFoundError: No module named 'datashader'

In [154]:
n=300
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}
scatters = {i: hv.Scatter(curves[i])  for i in range(n)}

walk = hv.NdOverlay(scatters).opts(
    hv.opts.Scatter(alpha=0.1, 
                    selection_color='red', 
                    color='black', 
                    tools=['box_select'], 
                    width=600)) 

walk_lines = hv.NdOverlay(curves).opts(
    hv.opts.Curve(
        alpha=0.05, selection_color='red', 
        hover_alpha=0.5, selection_alpha=0.5, 
        color='black', 
#         tools=['','hover'], 
        width=600)) 

selection = streams.Selection1D(source=walk)

walk_df = walk.dframe()



def selected_lines(index):
    global selected_events 
    selected_events = index
    if (index or len(index)>0):
        line_ids = list(walk_df.iloc[index]['Element'].unique()) 
    else:
        line_ids = []
        return walk * walk_lines
    chosen_lines = walk.select(Element=line_ids).opts( 
        hv.opts.Scatter(color='blue', alpha=0.3), 
        hv.opts.NdOverlay(show_legend=False))#, title=str(line_ids)))
    chosen_dots = walk_lines.select(Element=line_ids).opts( 
        hv.opts.Curve(color='blue', alpha=0.3), 
        hv.opts.NdOverlay(show_legend=False, title=str(len(line_ids))))
    return walk * chosen_dots * chosen_lines #* walk * walk_lines ## the order of the layout may make a difference

layout =  hv.DynamicMap(selected_lines, streams=[selection])
layout.opts(shared_datasource=True, active_tools=['box_select'])
(layout).opts(tools=['box_select'], width=800 )

:DynamicMap   []
   :Overlay
      .NdOverlay.I  :NdOverlay   [Element]
         :Scatter   [x]   (y)
      .NdOverlay.II :NdOverlay   [Element]
         :Curve   [x]   (y)

In [76]:
selection.index

[730, 890, 474]

In [200]:
n=300
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}
scatters = {i: hv.Scatter(curves[i])  for i in range(n)}

walk = hv.NdOverlay(scatters).opts(
    hv.opts.Scatter(alpha=0.1, 
                    selection_color='red', 
                    color='black', 
                    tools=['box_select'], 
                    width=600)) 

walk_lines = hv.NdOverlay(curves).opts(
    hv.opts.Curve(
        alpha=0.05, selection_color='red', 
        hover_alpha=0.5, selection_alpha=0.5, 
        color='black', 
#         tools=['','hover'], 
        width=600)) 

# selection = streams.Selection1D(source=walk)
box = streams.BoundsXY(source=walk, bounds=(0,0,0,0))

walk_df = walk.dframe()
walk_ds = hv.Dataset(walk_df)

def box_cb(bounds):
    global gbounds
    gbounds = bounds
    if bounds:
        x0, y0, x1, y1 = bounds
        sel_elements = walk_ds.select(x=(x0,x1),y=(y0,y1))['Element']
        sel_lines = walk_lines.select(Element=list(sel_elements)).opts( hv.opts.Curve(color='blue',alpha=0.4))
        return walk * sel_lines
    else:
        return walk
    

def selected_lines(index):
    global selected_events 
    selected_events = index
    if (index or len(index)>0):
        line_ids = list(walk_df.iloc[index]['Element'].unique()) 
    else:
        line_ids = []
        return walk * walk_lines
    chosen_lines = walk.select(Element=line_ids).opts( 
        hv.opts.Scatter(color='blue', alpha=0.3), 
        hv.opts.NdOverlay(show_legend=False))#, title=str(line_ids)))
    chosen_dots = walk_lines.select(Element=line_ids).opts( 
        hv.opts.Curve(color='blue', alpha=0.3), 
        hv.opts.NdOverlay(show_legend=False, title=str(len(line_ids))))
    return walk * chosen_dots * chosen_lines #* walk * walk_lines ## the order of the layout may make a difference

# layout =  hv.DynamicMap(selected_lines, streams=[selection])
blayout =  hv.DynamicMap(box_cb, streams=[box])
# layout.opts( active_tools=['box_select'])
(blayout).opts(active_tools=['box_select'], width=800, show_legend=False, )

:DynamicMap   []
   :Overlay
      .NdOverlay.I  :NdOverlay   [Element]
         :Scatter   [x]   (y)
      .NdOverlay.II :NdOverlay   [Element]

In [112]:
line_ids = list(walk.dframe().iloc[selection.index]['Element'])
walk * walk.select(Element= line_ids) * walk_lines.select(Element= line_ids).opts(hv.opts.Curve(alpha=.2))

:Overlay
   .NdOverlay.I   :NdOverlay   [Element]
      :Scatter   [x]   (y)
   .NdOverlay.II  :NdOverlay   [Element]
      :Scatter   [x]   (y)
   .NdOverlay.III :NdOverlay   [Element]
      :Curve   [x]   (y)

In [70]:
walk[(9,)].opts(color='red',alpha=1.0) * walk_lines[(9,)]

:Overlay
   .Scatter.I :Scatter   [x]   (y)
   .Curve.I   :Curve   [x]   (y)

In [70]:
walk[(9,)].opts(color='red',alpha=1.0) * walk_lines[(9,)]

:Overlay
   .Scatter.I :Scatter   [x]   (y)
   .Curve.I   :Curve   [x]   (y)

In [58]:
walk.data[(9,)].opts(color='red',alpha=1.0) * walk_lines[(9,)]

:Overlay
   .Scatter.I :Scatter   [x]   (y)
   .Curve.I   :Curve   [x]   (y)

In [63]:
walk_lines.select(Element=[0,5])

:NdOverlay   [Element]
   :Curve   [x]   (y)

In [52]:
selection.contents

{'index': [122,
  250,
  345,
  953,
  825,
  473,
  281,
  474,
  730,
  762,
  763,
  475,
  826,
  506,
  507,
  827,
  251,
  379,
  378,
  634,
  633,
  665,
  505,
  761,
  729,
  732,
  284,
  283,
  731,
  282,
  954,
  698,
  699,
  635,
  411,
  955,
  667,
  700,
  668,
  412,
  956,
  828,
  476,
  508,
  252,
  764,
  121,
  346,
  347,
  123,
  380,
  377,
  249]}

In [60]:
walk.dframe().iloc[selection.index]['Element']

26     3
26     7
25    10
25    29
25    25
25    14
25     8
26    14
26    22
26    23
27    23
27    14
26    25
26    15
27    15
27    25
27     7
27    11
26    11
26    19
25    19
25    20
25    15
25    23
25    22
28    22
28     8
27     8
27    22
26     8
26    29
26    21
27    21
27    19
27    12
27    29
27    20
28    21
28    20
28    12
28    29
28    25
28    14
28    15
28     7
28    23
25     3
26    10
27    10
27     3
28    11
25    11
25     7
Name: Element, dtype: int64

In [50]:
walk.data

OrderedDict([((0,), :Scatter   [x]   (y)),
             ((1,), :Scatter   [x]   (y)),
             ((2,), :Scatter   [x]   (y)),
             ((3,), :Scatter   [x]   (y)),
             ((4,), :Scatter   [x]   (y)),
             ((5,), :Scatter   [x]   (y)),
             ((6,), :Scatter   [x]   (y)),
             ((7,), :Scatter   [x]   (y)),
             ((8,), :Scatter   [x]   (y)),
             ((9,), :Scatter   [x]   (y)),
             ((10,), :Scatter   [x]   (y)),
             ((11,), :Scatter   [x]   (y)),
             ((12,), :Scatter   [x]   (y)),
             ((13,), :Scatter   [x]   (y)),
             ((14,), :Scatter   [x]   (y)),
             ((15,), :Scatter   [x]   (y)),
             ((16,), :Scatter   [x]   (y)),
             ((17,), :Scatter   [x]   (y)),
             ((18,), :Scatter   [x]   (y)),
             ((19,), :Scatter   [x]   (y)),
             ((20,), :Scatter   [x]   (y)),
             ((21,), :Scatter   [x]   (y)),
             ((22,), :Scatter   [x]   (y))

In [27]:
len(selection.index)

960

In [45]:
selection.index

[22]

In [16]:
[walk.data[i] for i in selection.index]

KeyError: 789

In [31]:
# select just selects sections of the plot to present, doesn't access data underneath.. 
walk_lines.select(y=(0,10))

:NdOverlay   [Element]
   :Curve   [x]   (y)

In [5]:
%%opts NdOverlay [legend_limit=0] Curve [tools=['tap']] (line_width=1 )
ndoverlay = hv.NdOverlay({i: hv.Curve(np.arange(10)*i) for i in range(5)})

selection = streams.Selection1D(source=ndoverlay)
dmap = hv.DynamicMap(lambda index: ndoverlay[index] if index else ndoverlay.clone(),
                     kdims=[], streams=[selection])
ndoverlay + dmap

:Layout
   .NdOverlay.I  :NdOverlay   [Element]
      :Curve   [x]   (y)
   .DynamicMap.I :DynamicMap   []
      :NdOverlay   [Element]
         :Curve   [x]   (y)

In [153]:
n=30
steps = 32
# create random walk data
pos = torch.empty(n, steps).uniform_(-1, 1).cumsum(1)

# make the first position 0
pos = pos - pos[:,0].unsqueeze(1)

# make curves and points
curves = {i: hv.Curve( (torch.arange(steps), pos[i,:])) for i in range(n)}#.opts(tools=['box_select','click'])
# scatters = {i: hv.Scatter( (torch.arange(steps), pos[i,:])) for i in range(n)}

# walk = hv.NdOverlay(curves).opts(hv.opts.Curve(alpha=0.1, color='black', tools=['xbox_select'])) 
# walk = hv.NdOverlay(scatters).opts(hv.opts.Scatter(alpha=0.1, selection_color='red', color='black', tools=['box_select',], width=600)) 
walk_lines = hv.NdOverlay(curves).opts(hv.opts.Curve(
    alpha=0.01, selection_color='red', hover_alpha=0.5, selection_alpha=0.5, color='black', tools=['box_select','lasso_select','tap','hover'], width=600)) 
# walk_lines = hv.NdOverlay(curves).opts(hv.opts.Curve(alpha=0.01, hover_alpha=0.5, color='black', tools=['xbox_select','hover'], width=600)) 
# walk * 
walk_lines

:NdOverlay   [Element]
   :Curve   [x]   (y)